In [1]:
#PASO_1
#Importamos un xml
#Exportamos una lista con [cont, lat, lon, ele ]

from xml.dom.minidom import parse
import xml.dom.minidom
import numpy as np


#DOMTree = xml.dom.minidom.parse("182k-bike-ironman-lanzarote-2012.xml")
DOMTree = xml.dom.minidom.parse("challenge-madrid.xml")

collection = DOMTree.documentElement

trkpts = collection.getElementsByTagName("trkpt")
list_track = []

count=0
for trkpt in trkpts:
    count +=1
    if trkpt.hasAttribute("lat"):
      lat = float(trkpt.getAttribute("lat"))
    if trkpt.hasAttribute("lon"):
      lon = float(trkpt.getAttribute("lon"))
    
    ele = float(trkpt.getElementsByTagName("ele")[0].firstChild.nodeValue)
    
    #time = trkpt.getElementsByTagName("time")[0].firstChild.nodeValue
    #fecha = str.split(time,"T")[0]
    #time_seg = int(str.split(str.replace(str.split(time,"T")[1],"Z",""),":")[0])*3600+int(str.split(str.replace(str.split(time,"T")[1],"Z",""),":")[1])*60+int(str.split(str.replace(str.split(time,"T")[1],"Z",""),":")[2])
   
    list_track.append([0, 0, 0, lat, lon, ele ])

print ("Puntos cargados: %d" %count)


#lista_distancia = calcula_distancia(list_track)

#lista_tramos(lista_distancia)


Puntos cargados: 8421


In [2]:
#PASO_2
#CREAMOS LA FUNCION QUE DADA UNA LISTA CALCULA LA DISTANCIA ENTRE PUNTOS DADA SU LAT Y LON
#Exportamos una lista con [time, fecha, time_seg, lat, lon, ele, dist, time_elapse, desnivel, acum_dis ]

def calcula_distancia(lista):
    from geopy.distance import distance

    from geopy import Point
    dist_array = []
    dist_array.append([lista[0][0],lista[0][1],lista[0][2],lista[0][3],lista[0][4],lista[0][5],0,0,0,0])
    acum_dist = 0
        
    i=1
    while i < len(lista):
        #print("Punto %d" %i)
        punto1 = Point(lista[i-1][3], lista[i-1][4])
        punto2 = Point(lista[i][3], lista[i][4])
        #print(punto1, punto2)
        dist = distance(punto1, punto2).m
        acum_dist = acum_dist + dist
        time_elapse = lista[i][2]-lista[i-1][2]
        desnivel = lista[i][5]-lista[i-1][5]
        #print("Distancia en km: %d" %dist)
        dist_array.append([lista[i][0],lista[i][1],lista[i][2],lista[i][3],lista[i][4],lista[i][5],dist,time_elapse,desnivel,acum_dist])
        i+=1
    return dist_array

In [3]:
#PASO_3
#DADA UNA LISTA CON [time, fecha, time_seg, lat, lon, ele, dist, time_elapse, desnivel, acum_dis ] ITERAMOS...
#

def lista_tramos(lista):
    import math as mt
    import pandas as pd
    from scipy.signal import savgol_filter

    df = pd.DataFrame(lista)

    df.columns = ["tiempo","fecha","time_seg","lat","lon","ele","dist","time_elapse","desnivel","acum_dist"]

    axis_x = df["acum_dist"]
    axis_y = df["ele"]
    
    #Aplicamos la funcion de filtro de señal para suavizar los tramos
    axis_yhat = savgol_filter(axis_y, 31, 2)
    df["new_ele"] = axis_yhat

    #Calculo de tramos continuos de subida-bajada
    
    tramos = [0]
    tramo = 0
    pendiente = 0
    pendiente_ant = 0

    i = 1
    print("Entramos en el While1 %d" % len(df.index))
    while i < len(df.index):
        #print("punto = %d y tramo = %d" % (i, tramo))
        pendiente = df.iloc[i]["new_ele"]-df.iloc[i-1]["new_ele"]
        
        if pendiente * pendiente_ant >=0:
            tramos.append(tramo)
        else:
            tramo +=1
            tramos.append(tramo)
        i+=1
        pendiente_ant = pendiente
      
    print("tramos : %d" % len(tramos))
    print("Salimos del While")
    df["tramo"] = tramos
    
    #Calculo de tramos de 100m
    
    tramos_100 = [0]
    tramo = 0
    acum_dist = 0
    i = 1
    
    print("Entramos en el While2 %d" % len(df.index))
    
    while i < len(df.index):
        acum_dist = df.iloc[i-1]["acum_dist"]
        tramo = mt.floor(acum_dist/100)
        tramos_100.append(tramo)
        i +=1
    print("tramos_100m : %d" % len(tramos_100))
    print("Salimos del While2")
    df["tramos_100"] = tramos_100
    
    print("Escribimos fichero")
    writer = pd.ExcelWriter('output.xlsx')
    df.to_excel(writer,'Hoja_1')
    writer.save()
    print("Fin")
            
    return


In [4]:
lista_distancia = calcula_distancia(list_track)

In [5]:
lista_tramos(lista_distancia)

Entramos en el While1 8421
tramos : 8421
Salimos del While
Entramos en el While2 8421
tramos_100m : 8421
Salimos del While2
Escribimos fichero
Fin
